# How to create a folium map?

##### Necessary libraries

In [ ]:
from branca.element import Template, MacroElement
from selenium import webdriver
import folium
import json
import numpy as np
import os
import pandas as pd
import random
import requests
import time

#### Create a folium map object (set center of map, start and maximum zoom)

In [ ]:
m = folium.Map(location=[example_latitude, example_longitude], zoom_start=12, max_zoom=14)

#### Create a marker for a specific point on the map

In [ ]:
popup = folium.Popup('popup_title <br/> popup_subtitle', max_width=300)
icon = folium.features.CustomIcon('image_url')

folium.Marker([example_latitude, example_longitude], icon=icon, tooltip='tooltip_text', popup=popup).add_to(m)

#### In case you want to show travel time zone for a specific map point, you can do it with isochrones. <br/> For travel distance zones you can use isodistance. <br/> Both options are available free with ISO4APP API. Zones 10 min and 10 km are available for free with a public licence key.

In [ ]:
# Example for isochrone 10 min

parameters_10 = {
    "type": "isochrone",
    "value": 600,
    "lat": example_latitude,
    "lng": example_longitude,
    "mobility": "motor_vehicle",
    "concavity": 8   #degree of precision by isochrone borders
}

response_10 = requests.get("http://www.iso4app.net/rest/1.3/isoline.geojson?licKey=87B7FB96-83DA-4FBD-A312-7822B96BB143"
                       , params = parameters_10)

if response_10.status_code==200:
    geom_10 = response_10.json()
    print('Data successfully loaded')
else:
    print('Error by data loading')

In [ ]:
# Add isochrone to the map with a specifict style options

style_function_10 = lambda x: {
    'color': 'darkorange',
    'weight':2,
    'fillOpacity':0.2,
    'opacity':0.8}

folium.GeoJson(geom_10['features'][2], name='Time travel zone 10 min', style_function=style_function_10).add_to(m)

#### You can also show some points around main point on the map. <br/>For the example I will use some random geocoordinates

In [ ]:
random.seed(123)

list1 = example_latitude+np.random.randn(500)/45
list2 = example_longitude+np.random.randn(500)/25
list3 = np.random.randint(0,4,500)

zipped = zip(list1, list2, list3)

points = pd.DataFrame(zipped, columns=['latitude', 'longitude', 'value'])

In [ ]:
feature_group = folium.FeatureGroup(name='name_of_group')

for row in points.iterrows():
    if(row[1][2]>2):
        marker_active = folium.CircleMarker([row[1][0], row[1][1]], fill=True, fill_color='black', 
                                            color='black', radius=3, weight=1, fill_opacity=0.5)
        feature_group_active.add_child(marker_active)
        
    elif(row[1][2]>1):
        marker_active = folium.CircleMarker([row[1][0], row[1][1]], fill=True, fill_color='gray', 
                                            color='black', radius=3, weight=1, fill_opacity=0.5)
        feature_group_active.add_child(marker_active)
        
    elif(row[1][2]>0):
        marker_active = folium.CircleMarker([row[1][0], row[1][1]], fill=True, fill_color='white', 
                                            color='black', radius=3, weight=1, fill_opacity=0.5)
        feature_group_active.add_child(marker_active)
        
    else:
        marker_inactive = folium.CircleMarker([row[1][0], row[1][1]], fill=True, fill_color='red', 
                                              color='red', radius=3, weight=1, fill_opacity=0.5)
        feature_group_inactive.add_child(marker_inactive)
        

m.add_child(feature_group)

#### For the control of the specified layers you can use below option

In [ ]:
folium.LayerControl().add_to(m)

#### Finally you can show the map

In [ ]:
m

#### What might be missing is the map legend. You can add one using the below code.

In [ ]:
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Folium map</title>
</head>
<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid rgba(0, 0, 0, 0.2); background-color:rgba(255, 255, 255, 1);
     border-radius:5px; padding: 10px; font-size:14px; right: 5px; bottom: 20px; height: auto; width: auto'>
     
<div class='legend-title'>Legend title</div>
<div class='legend-part-title'>Time travel zone</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:darkorange;opacity:0.5;'></span>10 min</li>
  </ul>
</div> 
<div class='legend-part-title2'>Points</div>  
<div class='legend-scale'>
  <ul class='legend-labels2'>
    <li><span style='background:white;opacity:0.6;'></span>value 1-2</li>
    <li><span style='background:gray;opacity:0.6;'></span>value 3-4</li>
    <li><span style='background:black;opacity:0.6;'></span>value 5+</li>
  </ul>
</div>
<div class='legend-part-title'> Another subtitle </div>  
<div class='legend-scale'>
  <ul class='legend-labels3'>
    <li><span style='background:red;opacity:0.6;'></span>Value 0</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-part-title {
    text-align: left;
    margin-bottom: 5px;
    font-size: 90%;
    }
  .maplegend .legend-part-title2 {
    text-align: left;
    margin-bottom: 5px;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    }
  .maplegend ul.legend-labels2 li span {
    display: block;
    float: left;
    height: 16px;
    width: 16px;
    margin-right: 5px;
    margin-left: 0;
    border-style:solid;
    border-width:1px;
    border-color:black;
    border-radius:50%
    }
  .maplegend ul.legend-labels3 li span {
    display: block;
    float: left;
    height: 16px;
    width: 16px;
    margin-right: 5px;
    margin-left: 0;
    border-style:solid;
    border-width:1px;
    border-color:red;
    border-radius:50%
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

m_legend = MacroElement()
m_legend._template = Template(template)

m.get_root().add_child(m_legend)
m

#### Saving the map in HTML file

In [ ]:
m.save('map.html')

#### Saving the map as PNG file (driver for web browser is necessary) <br/> For zooming we need some delay for the map to render

In [ ]:
html_path='file://{path}/map.html'.format(path=os.getcwd())

delay = 10

browser = webdriver.Chrome()
browser.maximize_window()

browser.get(html_path)
browser.save_screenshot('map.png')

zoom_in = browser.find_element_by_class_name('leaflet-control-zoom-in')
zoom_in.click()
time.sleep(delay)
browser.save_screenshot('map+zoom.png')

browser.quit()

## <hr>Enyjoy the code! <br/> Feel free to give your opinion or share your own experience with Folium :)